# Haystack 💙 Google Gemini

*by Tuana Celik: [Twitter](https://twitter.com/tuanacelik), [LinkedIn](https://www.linkedin.com/in/tuanacelik/), Tilde Thurium: [Twitter](https://twitter.com/annthurium), [LinkedIn](https://www.linkedin.com/in/annthurium/) and Silvano Cerza: [LinkedIn](https://www.linkedin.com/in/silvanocerza/)*

**📚 Check out the [Gemini Models with Google Vertex AI Integration for Haystack](https://haystack.deepset.ai/blog/gemini-models-with-google-vertex-for-haystack) article for a detailed run through of this example.**

This is a notebook showing how you can use Gemini with Haystack.

Gemini is Google's newest model. You can read more about its capabilities [here](https://deepmind.google/technologies/gemini/#capabilities).



## Install dependencies

As a prerequisite, you need to have a Google Cloud Project set up that has access to Gemini. Following that, you'll only need to authenticate yourself in this Colab.

First thing first we need to install our dependencies.


In [ ]:
!pip install --upgrade haystack-ai google-genai-haystack trafilatura

To use Gemini you need to have a Google Cloud Platform account and be logged in using Application Default Credentials (ADCs). For more info see the [official documentation](https://cloud.google.com/docs/authentication/provide-credentials-adc).

Time to login!

In [ ]:
from google.colab import auth

auth.authenticate_user()

Remember to set the `project_id` variable to a valid project ID that you have enough authorization to use for Gemini.
We're going to use this one throughout the example!

To find your project ID you can find it in the [GCP resource manager](https://console.cloud.google.com/cloud-resource-manager) or locally by running `gcloud projects list` in your terminal. For more info on the gcloud CLI see the [official documentation](https://cloud.google.com/cli).

In [ ]:
project_id = input("Enter your project ID:")

## Use `gemini-2.0-flash`

### Answer Questions

Now that we setup everything we can create an instance of our `GoogleGenAIChatGenerator`. This component supports both Gemini and Vertex AI. For this demo, we will set `api="vertex"`, and pass our project_id as vertex_ai_project. 

In [ ]:
from haystack_integrations.components.generators.googlegenai import GoogleGenAIChatGenerator

gemini = GoogleGenAIChatGenerator(model="gemini-2.0-flash", api="vertex", vertex_ai_project=project_id, vertex_ai_location="europe-west1")

Let's start by asking something simple.

This component expects a list of `ChatMessage` as input to the `run()` method. You can pass text or function calls through the messages.

In [ ]:
from haystack.dataclasses import ChatMessage
messages = [ChatMessage.from_user("What is the most interesting thing you know?")]
result = gemini.run(messages = messages)
for answer in result["replies"]:
    print(answer)

## Function Calling with `gemini-2.0-flash`


With `gemini-2.0-flash`, we can also use function calling!
So let's see how we can do that 👇

Let's see if we can build a system that can run a `get_current_weather` function, based on a question asked in natural language.

First we create our function definition and tool (learn more about [Tools](https://docs.haystack.deepset.ai/docs/tool) in the docs).

For demonstration purposes, we're simply creating a `get_current_weather` function that returns an object which will _always_ tell us it's 'Sunny, and 21.8 degrees'... If it's Celsius, that's a good day! ☀️

In [ ]:
from haystack.components.tools import ToolInvoker
from haystack.tools import create_tool_from_function
from typing import Annotated

def get_current_weather(
    location: Annotated[str, "The city for which to get the weather, e.g. 'San Francisco'"] = "Munich",
    unit: Annotated[str, "The unit for the temperature, e.g. 'celsius'"] = "celsius",
):
  return {"weather": "sunny", "temperature": 21.8, "unit": unit}

weather_tool = create_tool_from_function(get_current_weather)

In [ ]:

user_message = [ChatMessage.from_user("What is the temperature in celsius in Berlin?")]
replies = gemini.run(messages=user_message)["replies"]
print(replies)

Look at that! We go a message with some interesting information now.
We can use that information to call a real function locally.

Let's do exactly that and pass the result back to Gemini.

In [ ]:
tool_invoker = ToolInvoker(tools=[weather_tool])
tool_messages = tool_invoker.run(messages=replies)["tool_messages"]

messages = user_message + replies + tool_messages
print(messages)

res = gemini_chat.run(messages = messages)
res["replies"][0].text

Seems like the weather is nice and sunny, remember to put on your sunglasses. 😎

## Build a full Retrieval-Augmented Generation Pipeline with `gemini-2.0-flash`

As a final exercise, let's add the `GoogleGenAIChatGenerator` to a full RAG pipeline. In the example below, we are building a RAG pipeline that does question answering on the web, using `gemini-2.0-flash`

In [ ]:
from haystack.components.fetchers.link_content import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.rankers import TransformersSimilarityRanker
from haystack.components.builders.chat_prompt_builder import ChatPromptBuilder
from haystack import Pipeline

fetcher = LinkContentFetcher()
converter = HTMLToDocument()
document_splitter = DocumentSplitter(split_by="word", split_length=50)
similarity_ranker = TransformersSimilarityRanker(top_k=3)
gemini = GoogleGenAIChatGenerator(model="gemini-2.0-flash", api="vertex", vertex_ai_project=project_id, vertex_ai_location="europe-west1")

prompt_template = [ChatMessage.from_user("""
According to these documents:

{% for doc in documents %}
  {{ doc.content }}
{% endfor %}

Answer the given question: {{question}}
Answer:
""")]
prompt_builder = ChatPromptBuilder(template=prompt_template)

pipeline = Pipeline()
pipeline.add_component("fetcher", fetcher)
pipeline.add_component("converter", converter)
pipeline.add_component("splitter", document_splitter)
pipeline.add_component("ranker", similarity_ranker)
pipeline.add_component("prompt_builder", prompt_builder)
pipeline.add_component("gemini", gemini)

pipeline.connect("fetcher.streams", "converter.sources")
pipeline.connect("converter.documents", "splitter.documents")
pipeline.connect("splitter.documents", "ranker.documents")
pipeline.connect("ranker.documents", "prompt_builder.documents")
pipeline.connect("prompt_builder.prompt", "gemini")

Let's try asking Gemini to tell us about Haystack and how to use it.

In [ ]:
question = "What do graphs have to do with Haystack?"
result = pipeline.run({"prompt_builder": {"question": question},
                   "ranker": {"query": question},
                   "fetcher": {"urls": ["https://haystack.deepset.ai/blog/introducing-haystack-2-beta-and-advent"]}})

for answer in result["gemini"]["replies"]:
  print(answer)

Now you've seen some of what Gemini can do, as well as how to integrate it with Haystack. If you want to learn more:
- check out the Haystack [docs](https://docs.haystack.deepset.ai/docs) or [tutorials](https://haystack.deepset.ai/tutorials)
- Try out the [Gemini quickstart colab from Google](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb#scrollTo=IqFXdgDFRvlU)
- Participate in the [Advent of Haystack](https://haystack.deepset.ai/advent-of-haystack)